# Hydropower Simulations with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows and Sourabh Dalvi

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways.
This example demonstrates a few of the options for modeling hydropower generation.

## Dependencies

In [1]:
using SIIPExamples

### Modeling Packages

In [2]:
using InfrastructureSystems
const IS = InfrastructureSystems
using PowerSystems
const PSY = PowerSystems
using PowerSimulations
const PSI = PowerSimulations
using D3TypeTrees

### Data management packages

In [3]:
using Dates
using DataFrames

### Optimization packages

In [4]:
using JuMP
using Cbc #solver
Cbc_optimizer = JuMP.with_optimizer(Cbc.Optimizer, logLevel=1, ratioGap=0.5)

OptimizerFactory(Cbc.Optimizer, (), Base.Iterators.Pairs{Symbol,Real,Tuple{Symbol,Symbol},NamedTuple{(:logLevel, :ratioGap),Tuple{Int64,Float64}}}(:logLevel => 1,:ratioGap => 0.5))

### Logging
Using InfrastructureSystems, we can configure the console and file logging verbosity.

In [5]:
using Logging
logger = IS.configure_logging(console_level = Logging.Info,
                              file_level = Logging.Info,
                              filename = "op_problem_log.txt")

pkgpath = dirname(dirname(pathof(SIIPExamples)))

"/Users/cbarrows/Documents/repos/Examples"

### Data

In [6]:
include(joinpath(pathof(PSI), "../../test/test_utils/get_test_data.jl"))

hydro_generators5(nodes5) = [
                    HydroFix("HydroFix", true, nodes5[2], 0.0, 0.0,
                        TechHydro(0.600, PowerSystems.HY, (min = 0.0, max = 60.0), (min = 0.0, max = 60.0), nothing, nothing)
                    ),
                    HydroDispatch("HydroDispatch", true, nodes5[3], 0.0, 0.0,
                        TechHydro(0.600, PowerSystems.HY, (min = 0.0, max = 60.0), (min = 0.0, max = 60.0), (up = 10.0, down = 10.0), nothing),
                        TwoPartCost(15.0, 0.0), 10.0, 2.0, 5.0
                    )
                    ];

hydro_dispatch_timeseries_DA = [[TimeSeries.TimeArray(DayAhead,wind_ts_DA)],
                        [TimeSeries.TimeArray(DayAhead + Day(1),  wind_ts_DA)]];

hydro_timeseries_DA = [[TimeSeries.TimeArray(DayAhead,wind_ts_DA)],
                        [TimeSeries.TimeArray(DayAhead + Day(1),  wind_ts_DA)]];


hydro_timeseries_RT = [[TimeArray(RealTime,repeat(wind_ts_DA,inner=12))],
                     [TimeArray(RealTime + Day(1), repeat(wind_ts_DA,inner=12))]];

hydro_dispatch_timeseries_RT = [[TimeArray(RealTime,repeat(wind_ts_DA,inner=12))],
                     [TimeArray(RealTime + Day(1),  repeat(wind_ts_DA,inner=12))]];


c_sys5_hy = System(nodes, vcat(thermal_generators5_uc_testing(nodes), hydro_generators5(nodes), renewable_generators5(nodes)), loads5(nodes), branches5(nodes), nothing, 100.0, nothing, nothing)
for t in 1:2
   for (ix, l) in enumerate(get_components(PowerLoad, c_sys5_hy))
       add_forecast!(c_sys5_hy, l, Deterministic("get_maxactivepower", load_timeseries_DA[t][ix]))
   end
   for (ix, h) in enumerate(get_components(HydroDispatch, c_sys5_hy))
       add_forecast!(c_sys5_hy, h, Deterministic("get_rating", hydro_dispatch_timeseries_DA[t][ix]))
   end
   for (ix, h) in enumerate(get_components(HydroDispatch, c_sys5_hy))
       add_forecast!(c_sys5_hy, h, Deterministic("get_storage_capacity", hydro_dispatch_timeseries_DA[t][ix]))
   end
   for (ix, h) in enumerate(get_components(HydroFix, c_sys5_hy))
       add_forecast!(c_sys5_hy, h, Deterministic("get_rating", hydro_timeseries_DA[t][ix]))
   end
    for (ix, r) in enumerate(get_components(RenewableGen, c_sys5_hy))
        add_forecast!(c_sys5_hy, r, Deterministic("get_rating", ren_timeseries_DA[t][ix]))
    end
    for (ix, i) in enumerate(get_components(InterruptibleLoad, c_sys5_hy))
        add_forecast!(c_sys5_hy, i, Deterministic("get_maxactivepower", Iload_timeseries_DA[t][ix]))
    end
end

┌ Warning: Invalid range
│   valid_info.struct_name = "Bus"
│   field_name = "voltage"
│   field_value = 1.07
│   valid_range = "voltagelimits"
│   valid_info.ist_struct =
│    Bus 6 (Bus):
│       number: 6
│       name: Bus 6
│       bustype: PV
│       angle: -0.24818581963359368
│       voltage: 1.07
│       voltagelimits: (min = 0.94, max = 1.06)
│       basevoltage: 13.8
│       ext: Dict{String,Any}()
└ @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/Tjbyn/src/validation.jl:202
┌ Warning: Invalid range
│   valid_info.struct_name = "Bus"
│   field_name = "voltage"
│   field_value = 1.062
│   valid_range = "voltagelimits"
│   valid_info.ist_struct =
│    Bus 7 (Bus):
│       number: 7
│       name: Bus 7
│       bustype: PQ
│       angle: -0.23335052099164186
│       voltage: 1.062
│       voltagelimits: (min = 0.94, max = 1.06)
│       basevoltage: 13.8
│       ext: Dict{String,Any}()
└ @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/Tjbyn/src/vali

## Two PowerSimulations features determine hydropower representation.

### Hydropower `DeviceModel`s

First, the assignment of device formulations to particular device types gives us control
over the representation of devices. This is accomplished by defining `DeviceModel`
instances. For hydro power representations, we have two available generator types in
PowerSystems:

In [7]:
TypeTree(PSY.HydroGen)

HydroGen
├──HydroFix
└──HydroDispatch

And in PowerSimulations, we have several available formulations that can be applied to
the hydropower generation devices:

In [8]:
TypeTree(PSI.AbstractHydroFormulation)

PowerSimulations.AbstractHydroFormulation
├──PowerSimulations.AbstractHydroDispatchFormulation
│  ├──HydroDispatchReservoirFlow
│  ├──HydroDispatchRunOfRiver
│  └──HydroDispatchReservoirStorage
├──HydroFixed
└──PowerSimulations.AbstractHydroUnitCommitment
   ├──HydroCommitmentRunOfRiver
   ├──HydroCommitmentReservoirFlow
   └──HydroCommitmentReservoirStorage

Let's see what some of the different combinations create. First, let's apply the
`HydroDispatchRunOfRiver` formulation to the `HydroDispatch` generators, and the
`HydroFixed` formulation to `HydroFix` generators.

In [27]:
devices = Dict{Symbol, DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
                                    :Hyd2 =>DeviceModel(HydroFix, HydroFixed));

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

[ Info: The optimization model has no optimizer attached
[ Info: Instantiating the JuMP model
[ Info: Building HydroDispatch with HydroDispatchRunOfRiver formulation
[ Info: Building HydroFix with HydroFixed formulation
[ Info: Building CopperPlatePowerModel network formulation
[ Info: Building Objective



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroDispatchRunOfRiver
      Hyd2:
        device_type = HydroFix
        formulation = HydroFixed
  branches: 
  services: 


Now we can see the resulting JuMP model.

In [29]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Next, let's apply the `HydroDispatchReservoirFlow` formulation to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [30]:
devices = Dict{Symbol, DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirFlow),
                                    :Hyd2 =>DeviceModel(HydroFix, HydroFixed));

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 24)

[ Info: The optimization model has no optimizer attached
[ Info: Instantiating the JuMP model
[ Info: Building HydroDispatch with HydroDispatchReservoirFlow formulation
[ Info: Building HydroFix with HydroFixed formulation
[ Info: Building CopperPlatePowerModel network formulation
[ Info: Building Objective



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroDispatchReservoirFlow
      Hyd2:
        device_type = HydroFix
        formulation = HydroFixed
  branches: 
  services: 


In [31]:
PSI.model_time_steps(op_problem.psi_container)

1:24

Now we can see the resulting JuMP model.

In [25]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 24
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 24 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 48 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 24 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 24 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Next, let's apply the `HydroDispatchReservoirStorage` formulation to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [26]:
devices = Dict{Symbol, DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroDispatchReservoirStorage),
                                    :Hyd2 =>DeviceModel(HydroFix, HydroDispatchRunOfRiver));

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 24)

[ Info: The optimization model has no optimizer attached
[ Info: Instantiating the JuMP model
[ Info: Building HydroDispatch with HydroDispatchReservoirStorage formulation
[ Info: Setting DeviceEnergy initial conditions for the status of all devices HydroDispatch based on system data


ArgumentError: ArgumentError: forecast InfrastructureSystems.ForecastKey(InfrastructureSystems.DeterministicInternal, 2024-01-01T00:00:00, "get_inflow") is not stored

Now we can see the resulting JuMP model.

In [14]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 2
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

Finally, let's see the `HydroCommitmentReservoirFlow` formulation applied to the `HydroDispatch` generators, and the
`HydroDispatchRunOfRiver` formulation to `HydroFix` generators.

In [15]:
devices = Dict{Symbol, DeviceModel}(:Hyd1 => DeviceModel(HydroDispatch, HydroCommitmentReservoirStorage),
                                    :Hyd2 =>DeviceModel(HydroFix, HydroDispatchRunOfRiver));

template = PSI.OperationsProblemTemplate(CopperPlatePowerModel, devices, Dict(), Dict());

op_problem = PSI.OperationsProblem(GenericOpProblem, template, c_sys5_hy, horizon = 2)

[ Info: The optimization model has no optimizer attached
[ Info: Instantiating the JuMP model
[ Info: Building HydroDispatch with HydroCommitmentReservoirStorage formulation
[ Info: Setting DeviceStatus initial conditions for the status of all devices HydroDispatch based on system data
[ Info: Setting DevicePower initial conditions for the status of all devices HydroDispatch based on system data
[ Info: Setting TimeDurationON initial conditions for the status of all devices HydroDispatch based on system data
[ Info: Setting TimeDurationOFF initial conditions for the status of all devices HydroDispatch based on system data
[ Info: Building HydroFix with HydroDispatchRunOfRiver formulation
[ Info: Building CopperPlatePowerModel network formulation
[ Info: Building Objective



Operations Problem Specification

  transmission:  CopperPlatePowerModel
  devices: 
      Hyd1:
        device_type = HydroDispatch
        formulation = HydroCommitmentReservoirStorage
      Hyd2:
        device_type = HydroFix
        formulation = HydroDispatchRunOfRiver
  branches: 
  services: 


Now we can see the resulting JuMP model.

In [16]:
op_problem.psi_container.JuMPmodel

A JuMP Model
Minimization problem with:
Variables: 10
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 6 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 6 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*